## Código de red neuronal aprendiendo a jugar a piedra papel o tijera.

- difunde
- coopera
- aprende
- vuelve

-> simpre busca la solución mas sencilla.

In [85]:
options = ["piedra", "tijeras", "papel"]

In [117]:
def search_winner(p1, p2):
    if p1 == p2:
        result = 0
        
    elif p1 == "piedra" and p2 == "tijeras":
        result = 1
    elif p1 == "piedra" and p2 == "papel":
        result = 2
    elif p1 == "tijeras" and p2 == "piedra":
        result = 2
    elif p1 == "tijeras" and p2 == "papel":
        result = 1
    elif p1 == "papel" and p2 == "piedra":
        result = 1
    elif p1 == "papel" and p2 == "tijeras":
        result = 2
        
    return result    

In [87]:
search_winner("papel", "tijeras")

2

In [88]:
test = [
    ["piedra", "piedra", 0 ],
    ["piedra", "tijeras", 1],
    ["piedra", "papel", 2]
]

for partida in test:
    print(partida)
    print("Player1 %s Player2: %s Winner: %s Validation %s" % (
        partida[0], partida[1], search_winner(partida[0], partida[1]), partida[2]
    ))

['piedra', 'piedra', 0]
Player1 piedra Player2: piedra Winner: 0 Validation 0
['piedra', 'tijeras', 1]
Player1 piedra Player2: tijeras Winner: 1 Validation 1
['piedra', 'papel', 2]
Player1 piedra Player2: papel Winner: 2 Validation 2


In [89]:
from random import choice
def get_choice():
    return choice(options)

In [120]:
# es posible que falle esta parte ya que no buscamos todas las opciónes posibles
for i in range(10):
    player1 = get_choice()
    player2 = get_choice()
    print("Player1 %s Player2: %s Winner: %s" % (
        player1, player2, search_winner(player1, player2)))

Player1 : papel
Player2 : papel
Player1 papel Player2: papel Winner: 0
Player1 : papel
Player2 : tijeras
Player1 papel Player2: tijeras Winner: 2
Player1 : piedra
Player2 : tijeras
Player1 piedra Player2: tijeras Winner: 1
Player1 : papel
Player2 : papel
Player1 papel Player2: papel Winner: 0
Player1 : piedra
Player2 : papel
Player1 piedra Player2: papel Winner: 2
Player1 : papel
Player2 : tijeras
Player1 papel Player2: tijeras Winner: 2
Player1 : tijeras
Player2 : tijeras
Player1 tijeras Player2: tijeras Winner: 0
Player1 : tijeras
Player2 : piedra
Player1 tijeras Player2: piedra Winner: 2
Player1 : piedra
Player2 : tijeras
Player1 piedra Player2: tijeras Winner: 1
Player1 : papel
Player2 : tijeras
Player1 papel Player2: tijeras Winner: 2


## Vamos a convertir los datos de papel, piedra o tijera de datos 0 y 1

In [94]:
def str_to_list(option):
    if option=="piedra":
        res = [1, 0, 0]
    elif option=="tijeras":
        res = [0, 1, 0]
    elif option=="papel":
        res = [0, 0, 1]
    
    return res

data_X = list(map(str_to_list, ["piedra", "tijeras", "papel"]))
data_y = list(map(str_to_list, ["papel", "piedra", "tijeras"]))

print(data_X)
print(data_y)

[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
[[0, 0, 1], [1, 0, 0], [0, 1, 0]]


In [95]:
from sklearn.neural_network import MLPClassifier

In [145]:
clf = MLPClassifier(verbose=False, warm_start=True)

In [146]:
model = clf.fit([data_X[0]], [data_y[0]])
print(model)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=True)


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\neural_network\multilayer_perceptron.py:564: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


In [147]:
def play_and_learn(iters=10, debug=False):
    score = {"win": 0, "loose": 0}
    
    data_X = []
    data_y = []
    
    for i in range(iters):
        player1 = get_choice()
        predict = model.predict_proba([str_to_list(player1)])[0]

        if predict[0] >= 0.95:
            player2 = options[0]  
        elif predict[1] >= 0.95:
            player2 = options[1] 
        elif predict[2] >= 0.95:
            player2 = options[2]
        else:
            player2 = get_choice()
        
        if debug==True:
            print("Player1: %s Player2: %s Modelo: %s -->" % (player1, predict, player2))
        
        winner = search_winner(player1, player2)

        if debug==True:
            print("Comprobamos: p1 VS p2: %s" % winner)
            
        if winner==2:
            data_X.append(str_to_list(player1))
            data_y.append(str_to_list(player2))
            score["win"]= score["win"] + 1
        else:
            score["loose"] = score["loose"] + 1
        
    return score, data_X, data_y
        

In [148]:
score, data_X, data_y = play_and_learn(1, debug=True)
print(data_X)
print(data_y)
print("score: %s el porciento %s %%" % (score, (score["win"]*100/(score["win"]+score["loose"]))))
if len(data_X):
    model = model.partial_fit(data_X, data_y)

Player1: tijeras Player2: [0.97996714 0.03019889 0.03392681] Modelo: piedra -->
Comprobamos: p1 VS p2: 2
[[0, 1, 0]]
[[1, 0, 0]]
score: {'win': 1, 'loose': 0} el porciento 100.0 %


In [149]:
i = 0
historico_pct = []
while True:
    i+=1
    score, data_X, data_y = play_and_learn(iters=1000, debug=False)
    pct = (score["win"]*100/(score["win"]+score["loose"]))
    historico_pct.append(pct)
    print("Iter: %s - score: %s %s %%" % (i, score, pct))
    
    if len(data_X):
        model = model.partial_fit(data_X, data_y)
    
    if sum(historico_pct[-9:])==900:
        break


Iter: 1 - score: {'win': 522, 'loose': 478} 52.2 %
Iter: 2 - score: {'win': 567, 'loose': 433} 56.7 %
Iter: 3 - score: {'win': 551, 'loose': 449} 55.1 %
Iter: 4 - score: {'win': 551, 'loose': 449} 55.1 %
Iter: 5 - score: {'win': 548, 'loose': 452} 54.8 %
Iter: 6 - score: {'win': 551, 'loose': 449} 55.1 %
Iter: 7 - score: {'win': 531, 'loose': 469} 53.1 %
Iter: 8 - score: {'win': 565, 'loose': 435} 56.5 %
Iter: 9 - score: {'win': 562, 'loose': 438} 56.2 %
Iter: 10 - score: {'win': 557, 'loose': 443} 55.7 %
Iter: 11 - score: {'win': 555, 'loose': 445} 55.5 %
Iter: 12 - score: {'win': 559, 'loose': 441} 55.9 %
Iter: 13 - score: {'win': 354, 'loose': 646} 35.4 %
Iter: 14 - score: {'win': 319, 'loose': 681} 31.9 %
Iter: 15 - score: {'win': 346, 'loose': 654} 34.6 %
Iter: 16 - score: {'win': 340, 'loose': 660} 34.0 %
Iter: 17 - score: {'win': 322, 'loose': 678} 32.2 %
Iter: 18 - score: {'win': 358, 'loose': 642} 35.8 %
Iter: 19 - score: {'win': 305, 'loose': 695} 30.5 %
Iter: 20 - score: {'w

Iter: 159 - score: {'win': 523, 'loose': 477} 52.3 %
Iter: 160 - score: {'win': 568, 'loose': 432} 56.8 %
Iter: 161 - score: {'win': 549, 'loose': 451} 54.9 %
Iter: 162 - score: {'win': 555, 'loose': 445} 55.5 %
Iter: 163 - score: {'win': 586, 'loose': 414} 58.6 %
Iter: 164 - score: {'win': 578, 'loose': 422} 57.8 %
Iter: 165 - score: {'win': 557, 'loose': 443} 55.7 %
Iter: 166 - score: {'win': 556, 'loose': 444} 55.6 %
Iter: 167 - score: {'win': 552, 'loose': 448} 55.2 %
Iter: 168 - score: {'win': 544, 'loose': 456} 54.4 %
Iter: 169 - score: {'win': 565, 'loose': 435} 56.5 %
Iter: 170 - score: {'win': 555, 'loose': 445} 55.5 %
Iter: 171 - score: {'win': 554, 'loose': 446} 55.4 %
Iter: 172 - score: {'win': 556, 'loose': 444} 55.6 %
Iter: 173 - score: {'win': 544, 'loose': 456} 54.4 %
Iter: 174 - score: {'win': 558, 'loose': 442} 55.8 %
Iter: 175 - score: {'win': 553, 'loose': 447} 55.3 %
Iter: 176 - score: {'win': 752, 'loose': 248} 75.2 %
Iter: 177 - score: {'win': 1000, 'loose': 0} 1